<a href="https://colab.research.google.com/github/phharrison/pix2pix-runway/blob/master/pix2pix_runway_train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

Run these commands first. If you recieve a prompt asking you to restart the model, make sure to do so. The first command will take a few minutes. 

In [1]:
!git clone https://github.com/phharrison/pix2pix-runway

Cloning into 'pix2pix-runway'...
remote: Enumerating objects: 2081, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 2081 (delta 1), reused 0 (delta 0), pack-reused 2073
Receiving objects: 100% (2081/2081), 7.95 MiB | 11.72 MiB/s, done.
Resolving deltas: 100% (1362/1362), done.


In [2]:
import os
os.chdir('pix2pix-runway/')

In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 43.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 KB 26.5 MB/s eta 0:00:00
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408217 sha256=a7baeb304f7afe73d10830c80f65f6d84954db74928e03e155fb615fe471a7b5
  Stored in directory: /root/.cache/pip/wheels/58/9e/14/30f7cc4dafdd4d602fb00ca33c6edd1424fc0f5df10a02e060
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=5c109f29144357249b1e5d3be585055d65b9ad0d9d823c29731405096f5f3513
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3

# Dataset

Place your images into the `pix2pix_dataset` template folder using the following scheme: in `/pix2pix_dataset/A/train`, put training images in your input style. In `/pix2pix_dataset/B/train`, put the corresponding images in your output style. Place images that you want to convert from A to B into BOTH `/pix2pix_dataset/A/test` and `/pix2pix_dataset/B/test`. 

Technically, the `/pix2pix_dataset/B/test` directory is intended to allow you to compare the fake outputs of B to a real set of outputs, but you will not always have a real B to compare to the fake B. The notebook will not run without having images in `/pix2pix_dataset/B/test`, so copying the images you'd like to convert from `/pix2pix_dataset/A/test` to `/pix2pix_dataset/B/test` is a good workaround. 

Corresponding images in a pair {A,B} must be the same size and have the same filename, e.g., `/pix2pix_dataset/A/train/1.jpg` is considered to correspond to `/pix2pix_dataset/B/train/1.jpg`.

Once you have the images in the correct folders, zip the the `pix2pix_dataset` folder and upload the zipped file using the next cell:



In [4]:
from google.colab import files

uploaded = files.upload()

Saving pix2pix_dataset.zip to pix2pix_dataset.zip


Next, we'll unzip the folder to the `/datasets/` directory:

In [5]:
!unzip ./pix2pix_dataset.zip -d ./datasets/

Archive:  ./pix2pix_dataset.zip
   creating: ./datasets/pix2pix_dataset/
   creating: ./datasets/pix2pix_dataset/A/
   creating: ./datasets/pix2pix_dataset/A/test/
  inflating: ./datasets/pix2pix_dataset/A/test/file (1).png  
  inflating: ./datasets/pix2pix_dataset/A/test/file (10).png  
  inflating: ./datasets/pix2pix_dataset/A/test/file (11).png  
  inflating: ./datasets/pix2pix_dataset/A/test/file (2).png  
  inflating: ./datasets/pix2pix_dataset/A/test/file (3).png  
  inflating: ./datasets/pix2pix_dataset/A/test/file (4).png  
  inflating: ./datasets/pix2pix_dataset/A/test/file (5).png  
  inflating: ./datasets/pix2pix_dataset/A/test/file (6).png  
  inflating: ./datasets/pix2pix_dataset/A/test/file (7).png  
  inflating: ./datasets/pix2pix_dataset/A/test/file (8).png  
  inflating: ./datasets/pix2pix_dataset/A/test/file (9).png  
   creating: ./datasets/pix2pix_dataset/A/train/
  inflating: ./datasets/pix2pix_dataset/A/train/file (1).png  
  inflating: ./datasets/pix2pix_dataset/

# Pairing the Dataset

Run the next command to combine each pair of images (A,B) into a single image file, ready for training.

In [6]:
!python datasets/combine_A_and_B.py --fold_A ./datasets/pix2pix_dataset/A/ --fold_B ./datasets/pix2pix_dataset/B/ --fold_AB ./datasets/pix2pix_dataset/

[fold_A] =  ./datasets/pix2pix_dataset/A/
[fold_B] =  ./datasets/pix2pix_dataset/B/
[fold_AB] =  ./datasets/pix2pix_dataset/
[num_imgs] =  1000000
[use_AB] =  False
split = train, use 297/297 images
split = train, number of images = 297
split = test, use 11/11 images
split = test, number of images = 11


# Training

Run this cell to start training pix2pix on your combined images. You may see a number of `Cannot assign requested address` errors to begin the command, which are the result of running PyTorch in Google Colab. You can ignore the errors and should see the training epochs start to appear.

The model will save itself every 5 epochs. You can stop it at any time by pressing the 'stop' button (if that doesn't work, try right-clicking on the button and interrupting the execution), but more training will result in a better-performing model. 





In [ ]:
!python train.py --dataroot ./datasets/pix2pix_dataset --name pix2pix_dataset --model pix2pix --direction AtoB

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./datasets/pix2pix_dataset    	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              

# Testing

This cell will convert the images that you put in the `/pix2pix_dataset/A/test` directory from style A to style B. 

In [ ]:
!python test.py --dataroot ./datasets/pix2pix_dataset/ --direction AtoB --model pix2pix --name pix2pix_dataset --checkpoints_dir checkpoints_2

# Visualize

This will display a sample output from your neural network:

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/pix2pix_dataset/test_latest/images/0_fake_B.png')
plt.imshow(img)

You can run this cell to see the corresponding input image:

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/pix2pix_dataset/test_latest/images/0_real_A.png')
plt.imshow(img)

# Download
First, compress the results folder:

In [ ]:
!zip -r ./results.zip ./results/

Next, download that folder to your computer:

In [ ]:
from google.colab import files

files.download('./results.zip')